<a href="https://colab.research.google.com/github/eaa1648/school_projects/blob/main/_downloads/char_rnn_classification_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline


Classifying Names with a Character-Level RNN
*********************************************
**Author**: `Sean Robertson <https://github.com/spro/practical-pytorch>`_

We will be building and training a basic character-level RNN to classify
words. A character-level RNN reads words as a series of characters -
outputting a prediction and "hidden state" at each step, feeding its
previous hidden state into each next step. We take the final prediction
to be the output, i.e. which class the word belongs to.

Specifically, we'll train on a few thousand surnames from 18 languages
of origin, and predict which language a name is from based on the
spelling:

::

    $ python predict.py Hinton
    (-0.47) Scottish
    (-1.52) English
    (-3.57) Irish

    $ python predict.py Schmidhuber
    (-0.19) German
    (-2.48) Czech
    (-2.68) Dutch


**Recommended Reading:**

I assume you have at least installed PyTorch, know Python, and
understand Tensors:

-  http://pytorch.org/ For installation instructions
-  :doc:`/beginner/deep_learning_60min_blitz` to get started with PyTorch in general
-  :doc:`/beginner/pytorch_with_examples` for a wide and deep overview
-  :doc:`/beginner/former_torchies_tutorial` if you are former Lua Torch user

It would also be useful to know about RNNs and how they work:

-  `The Unreasonable Effectiveness of Recurrent Neural
   Networks <http://karpathy.github.io/2015/05/21/rnn-effectiveness/>`__
   shows a bunch of real life examples
-  `Understanding LSTM
   Networks <http://colah.github.io/posts/2015-08-Understanding-LSTMs/>`__
   is about LSTMs specifically but also informative about RNNs in
   general

Preparing the Data
==================

.. Note::
   Download the data from
   `here <https://download.pytorch.org/tutorial/data.zip>`_
   and extract it to the current directory.

Included in the ``data/names`` directory are 18 text files named as
"[Language].txt". Each file contains a bunch of names, one name per
line, mostly romanized (but we still need to convert from Unicode to
ASCII).

We'll end up with a dictionary of lists of names per language,
``{language: [names ...]}``. The generic variables "category" and "line"
(for language and name in our case) are used for later extensibility.



In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os

# Belirtilen bir dosya yolundaki tüm dosyaları bulan bir fonksiyon
def findFiles(path): return glob.glob(path)

# 'data/names/' klasöründeki tüm .txt dosyalarını bul ve listele
print(findFiles('data/names/*.txt'))

import unicodedata
import string

# ASCII harfler ve bazı özel karakterler; isim verisi temizliği için kullanılacak
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)


# Unicode karakterleri ASCII'ye dönüştüren fonksiyon
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
        and c in all_letters
    )

# Örnek bir isimde Unicode'dan ASCII'ye dönüşümün çıktısını gösterir
print(unicodeToAscii('Ślusàrski'))

# Her bir kategoriye (ülkeye) ait isimlerin tutulacağı sözlük
category_lines = {}
# Tüm kategorilerin (ülkelerin) tutulacağı liste
all_categories = []


# Belirtilen dosyadan satırları okuyan fonksiyon
def readLines(filename):
    # Dosyadaki her satırı okuyup Unicode'dan ASCII'ye dönüştürür
    lines = open(filename, encoding='utf-8').read().strip().split('\n')
    return [unicodeToAscii(line) for line in lines]

# 'data/names/' klasöründeki her dosya için işlemler
for filename in findFiles('data/names/*.txt'):
    # Dosya adından kategori adını çıkar (örneğin, İngilizce için 'English')
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)  # Kategori listesini güncelle
    lines = readLines(filename)      # Dosyadaki isimleri oku
    category_lines[category] = lines  # Kategoriye ait isimleri sözlüğe ekle

# Kategori (ülke) sayısını belirle
n_categories = len(all_categories)


Now we have ``category_lines``, a dictionary mapping each category
(language) to a list of lines (names). We also kept track of
``all_categories`` (just a list of languages) and ``n_categories`` for
later reference.




In [ ]:
# 'Italian' anahtarı altında bulunan ilk 5 ismi yazdır
print(category_lines['Italian'][:5])


Turning Names into Tensors
--------------------------

Now that we have all the names organized, we need to turn them into
Tensors to make any use of them.

To represent a single letter, we use a "one-hot vector" of size
``<1 x n_letters>``. A one-hot vector is filled with 0s except for a 1
at index of the current letter, e.g. ``"b" = <0 1 0 0 0 ...>``.

To make a word we join a bunch of those into a 2D matrix
``<line_length x 1 x n_letters>``.

That extra 1 dimension is because PyTorch assumes everything is in
batches - we're just using a batch size of 1 here.




In [ ]:
import torch

# all_letters dizisinde belirtilen bir harfin indeksini bulan fonksiyon, örneğin "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)

# Bir harfi <1 x n_letters> boyutunda bir tensöre çevirme fonksiyonu (örnek için)
# Bu tensör bir "one-hot" vektörüdür, sadece belirli bir harfin konumunda 1 olur
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)  # <1 x n_letters> boyutunda sıfırlarla dolu bir tensör oluşturur
    tensor[0][letterToIndex(letter)] = 1  # Harfin indeksine göre tensörde 1 olarak işaretler
    return tensor

# Bir metni <line_length x 1 x n_letters> boyutunda tensöre çevirme fonksiyonu
# Bu tensör, harfleri "one-hot" vektörleri olarak temsil eden bir dizi içerir
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)  # Her harf için sıfırlardan oluşan bir tensör oluşturur
    for li, letter in enumerate(line):  # Metindeki her harf için
        tensor[li][0][letterToIndex(letter)] = 1  # Harfin indeksine göre tensörde 1 olarak işaretler
    return tensor

# 'J' harfini "one-hot" tensör olarak yazdır
print(letterToTensor('J'))

# 'Jones' kelimesini tensör boyutunda yazdır
print(lineToTensor('Jones').size())


Creating the Network
====================

Before autograd, creating a recurrent neural network in Torch involved
cloning the parameters of a layer over several timesteps. The layers
held hidden state and gradients which are now entirely handled by the
graph itself. This means you can implement a RNN in a very "pure" way,
as regular feed-forward layers.

This RNN module (mostly copied from `the PyTorch for Torch users
tutorial <http://pytorch.org/tutorials/beginner/former_torchies/
nn_tutorial.html#example-2-recurrent-net>`__)
is just 2 linear layers which operate on an input and hidden state, with
a LogSoftmax layer after the output.

.. figure:: https://i.imgur.com/Z2xbySO.png
   :alt:





In [ ]:
import torch.nn as nn

# RNN sınıfını tanımlıyoruz, nn.Module sınıfını miras alıyor
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()  # nn.Module'ın özelliklerini RNN'e aktarır

        self.hidden_size = hidden_size  # Gizli katmanın boyutu

        # Giriş ve gizli durumun birleşiminden gizli duruma geçiş katmanı
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)

        # Giriş ve gizli durumun birleşiminden çıkışa geçiş katmanı
        self.i2o = nn.Linear(input_size + hidden_size, output_size)

        # Çıkış katmanı için LogSoftmax aktivasyon fonksiyonu
        self.softmax = nn.LogSoftmax(dim=1)

    # İleri yayılım fonksiyonu, RNN'in giriş ve gizli durumu işleyerek çıktıyı üretmesini sağlar
    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)  # Giriş ve gizli durumu birleştiriyoruz
        hidden = self.i2h(combined)               # Yeni gizli durumu hesaplıyoruz
        output = self.i2o(combined)               # Çıkış hesaplanıyor
        output = self.softmax(output)             # LogSoftmax ile normalize ediliyor
        return output, hidden                     # Çıktı ve yeni gizli durum döndürülüyor

    # Başlangıç gizli durumu oluşturma fonksiyonu (başlangıçta tüm değerler 0)
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

# Gizli katman boyutu
n_hidden = 128

# RNN modelini başlatıyoruz, giriş boyutu n_letters, çıkış boyutu n_categories
rnn = RNN(n_letters, n_hidden, n_categories)


To run a step of this network we need to pass an input (in our case, the
Tensor for the current letter) and a previous hidden state (which we
initialize as zeros at first). We'll get back the output (probability of
each language) and a next hidden state (which we keep for the next
step).




In [ ]:
# 'A' harfi için bir "one-hot" tensör oluşturuyoruz
input = letterToTensor('A')

# Gizli durumun başlangıç değeri olarak sıfırlardan oluşan bir tensör oluşturuyoruz
hidden = torch.zeros(1, n_hidden)

# RNN modeline 'A' harfi ve başlangıç gizli durumunu vererek çıktıyı ve bir sonraki gizli durumu elde ediyoruz
output, next_hidden = rnn(input, hidden)


For the sake of efficiency we don't want to be creating a new Tensor for
every step, so we will use ``lineToTensor`` instead of
``letterToTensor`` and use slices. This could be further optimized by
pre-computing batches of Tensors.




In [ ]:
# 'Albert' ismini temsil eden bir tensör oluşturuyoruz.
# Bu tensör, her harfi "one-hot" olarak temsil eden bir dizi tensörden oluşur.
input = lineToTensor('Albert')

# Gizli durumun başlangıç değeri olarak sıfırlardan oluşan bir tensör oluşturuyoruz
hidden = torch.zeros(1, n_hidden)

# İlk harfi (input[0]) ve gizli durumu RNN modeline vererek çıktıyı ve yeni gizli durumu alıyoruz
output, next_hidden = rnn(input[0], hidden)

# Çıktıyı ekrana yazdırıyoruz; bu çıktı, modelin 'A' harfi ve başlangıç gizli durumu ile oluşturduğu tahmini temsil eder
print(output)


As you can see the output is a ``<1 x n_categories>`` Tensor, where
every item is the likelihood of that category (higher is more likely).




Training
========
Preparing for Training
----------------------

Before going into training we should make a few helper functions. The
first is to interpret the output of the network, which we know to be a
likelihood of each category. We can use ``Tensor.topk`` to get the index
of the greatest value:




In [ ]:
# Modelin çıktısından kategori tahminini belirleyen fonksiyon
def categoryFromOutput(output):
    # En yüksek değeri ve onun indeksini buluyoruz (bu kategori tahminidir)
    top_n, top_i = output.topk(1)  # En yüksek olasılık değerine sahip olanı seçiyoruz
    category_i = top_i[0].item()   # İndeksi bir Python tamsayısına çeviriyoruz
    return all_categories[category_i], category_i  # Kategori adını ve indeksini döndürüyoruz

# Tahmin edilen kategori ve indeksini ekrana yazdırıyoruz
print(categoryFromOutput(output))


We will also want a quick way to get a training example (a name and its
language):




In [ ]:
import random

# Listeden rastgele bir öğe seçen fonksiyon
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

# Rastgele bir eğitim örneği oluşturan fonksiyon
def randomTrainingExample():
    # Kategoriler arasından rastgele bir kategori seçiyoruz
    category = randomChoice(all_categories)

    # Seçilen kategoriye ait isimler arasından rastgele bir isim seçiyoruz
    line = randomChoice(category_lines[category])

    # Kategori için indeks tensörünü oluşturuyoruz (kategori numarasını uzunluk türünde tensor yapıyoruz)
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)

    # Seçilen ismi (line) tensöre çeviriyoruz
    line_tensor = lineToTensor(line)

    # Kategori adı, isim, kategori tensörü ve isim tensörünü döndürüyoruz
    return category, line, category_tensor, line_tensor

# 10 adet rastgele eğitim örneği oluşturup yazdırıyoruz
for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print('category =', category, '/ line =', line)


Training the Network
--------------------

Now all it takes to train this network is show it a bunch of examples,
have it make guesses, and tell it if it's wrong.

For the loss function ``nn.NLLLoss`` is appropriate, since the last
layer of the RNN is ``nn.LogSoftmax``.




In [ ]:
# Negatif Log-Likelihood (NLL) kayıp fonksiyonunu tanımlıyoruz
criterion = nn.NLLLoss()


Each loop of training will:

-  Create input and target tensors
-  Create a zeroed initial hidden state
-  Read each letter in and

   -  Keep hidden state for next letter

-  Compare final output to target
-  Back-propagate
-  Return the output and loss




In [ ]:
# Öğrenme oranını tanımlıyoruz; bu değer çok yüksek olursa model kararsızlaşabilir,
# çok düşük olursa model öğrenim sürecini etkili bir şekilde gerçekleştiremeyebilir
learning_rate = 0.005

# Modelin eğitimini gerçekleştiren fonksiyon
def train(category_tensor, line_tensor):
    # RNN için başlangıç gizli durumunu başlatıyoruz
    hidden = rnn.initHidden()

    # RNN modelinin gradyanlarını sıfırlıyoruz
    rnn.zero_grad()

    # Girdi tensöründeki her harf için RNN modelini çalıştırıyoruz
    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    # Çıktıyı ve kategori tensörünü kullanarak kaybı hesaplıyoruz
    loss = criterion(output, category_tensor)

    # Kayıp değerine göre gradyanları hesaplıyoruz
    loss.backward()

    # Öğrenme oranı ile çarpılan gradyanları, parametre değerlerine ekliyoruz
    # Bu işlem, her parametrenin güncellenmesini sağlar
    for p in rnn.parameters():
        p.data.add_(-learning_rate, p.grad.data)

    # Son çıktıyı ve kayıp değerini döndürüyoruz
    return output, loss.item()


Now we just have to run that with a bunch of examples. Since the
``train`` function returns both the output and loss we can print its
guesses and also keep track of loss for plotting. Since there are 1000s
of examples we print only every ``print_every`` examples, and take an
average of the loss.




In [ ]:
import time
import math

# Toplam iterasyon sayısı
n_iters = 100000
# Her 5000 iterasyonda çıktı vermek için ayarlama
print_every = 5000
# Her 1000 iterasyonda kayıp değerini grafiğe eklemek için ayarlama
plot_every = 1000

# Kayıpları takip etmek için bir liste
current_loss = 0
all_losses = []

# Geçen süreyi hesaplayan fonksiyon
def timeSince(since):
    now = time.time()  # Şu anki zamanı al
    s = now - since    # Başlangıç zamanından bu yana geçen süreyi hesapla
    m = math.floor(s / 60)  # Dakika cinsinden süreyi hesapla
    s -= m * 60  # Saniye cinsinden süreyi hesapla
    return '%dm %ds' % (m, s)  # Dakika ve saniyeyi döndür

start = time.time()  # Eğitim süresini başlat

# Eğitim döngüsü
for iter in range(1, n_iters + 1):
    # Rastgele bir eğitim örneği oluştur
    category, line, category_tensor, line_tensor = randomTrainingExample()
    # Modeli eğit ve kaybı hesapla
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss  # Geçerli kaybı güncelle

    # Belirli bir iterasyonda çıktı ver
    if iter % print_every == 0:
        guess, guess_i = categoryFromOutput(output)  # Modelin tahminini al
        # Doğruluğu kontrol et
        correct = '✓' if guess == category else '✗ (%s)' % category
        # İterasyon numarasını, kaybı ve tahmini yazdır
        print('%d %d%% (%s) %.4f %s / %s %s' % (
            iter, iter / n_iters * 100, timeSince(start), loss, line, guess, correct))

    # Geçerli kayıp ortalamasını kayıplar listesine ekle
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)  # Kayıp ortalamasını ekle
        current_loss = 0  # Geçerli kaybı sıfırla


Plotting the Results
--------------------

Plotting the historical loss from ``all_losses`` shows the network
learning:




In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Yeni bir grafik oluştur
plt.figure()
# Kayıp değerlerini grafiğe çiz
plt.plot(all_losses)

# Eksenleri ayarlamak için ticker kullan
plt.xlabel('Iterasyon (her 1000\'de bir)')
plt.ylabel('Kayıp (Loss)')
plt.title('Eğitim Kaybı Grafiği')

# Y ekseninde sayıları daha iyi göstermek için bir format belirle
plt.gca().yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))

# Grafiği göster
plt.show()


Evaluating the Results
======================

To see how well the network performs on different categories, we will
create a confusion matrix, indicating for every actual language (rows)
which language the network guesses (columns). To calculate the confusion
matrix a bunch of samples are run through the network with
``evaluate()``, which is the same as ``train()`` minus the backprop.




In [ ]:
# Doğru tahminleri takip etmek için bir karmaşa matrisini başlat
confusion = torch.zeros(n_categories, n_categories)  # n_categories x n_categories boyutunda sıfırlardan oluşan bir matris
n_confusion = 10000  # Değerlendirme için kullanılacak örnek sayısı

# Bir satıra verilen çıktıyı döndürmek için fonksiyon
def evaluate(line_tensor):
    hidden = rnn.initHidden()  # RNN'in gizli durumunu başlat

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)  # Her harf için çıkışı hesapla

    return output  # Son çıkışı döndür

# Birçok örneği geçerek hangi tahminlerin doğru yapıldığını kaydet
for i in range(n_confusion):
    category, line, category_tensor, line_tensor = randomTrainingExample()  # Rastgele bir eğitim örneği al
    output = evaluate(line_tensor)  # Örneği değerlendir
    guess, guess_i = categoryFromOutput(output)  # Tahmin edilen kategoriyi al
    category_i = all_categories.index(category)  # Gerçek kategorinin indeksini bul
    confusion[category_i][guess_i] += 1  # Karmaşa matrisinde doğru tahmini güncelle

# Her satırı kendi toplamına bölerek normalize et
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()  # Satır toplamına böl

# Grafik ayarları
fig = plt.figure()  # Yeni bir figür oluştur
ax = fig.add_subplot(111)  # 1x1'lik bir ızgarada ilk alt grafiği oluştur
cax = ax.matshow(confusion.numpy())  # Karmaşa matrisini görselleştir
fig.colorbar(cax)  # Renk çubuğunu ekle

# Eksen ayarları
ax.set_xticklabels([''] + all_categories, rotation=90)  # X eksenindeki etiketleri ayarla
ax.set_yticklabels([''] + all_categories)  # Y eksenindeki etiketleri ayarla

# Her tikte etiket zorla
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))  # X eksenindeki her bir tike bir etiket yerleştir
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))  # Y eksenindeki her bir tike bir etiket yerleştir

# Grafiği göster
plt.show()


You can pick out bright spots off the main axis that show which
languages it guesses incorrectly, e.g. Chinese for Korean, and Spanish
for Italian. It seems to do very well with Greek, and very poorly with
English (perhaps because of overlap with other languages).




Running on User Input
---------------------




In [ ]:
def predict(input_line, n_predictions=3):
    # Kullanıcıdan alınan girdi satırını yazdır
    print('\n> %s' % input_line)
    with torch.no_grad():  # Gradyan hesaplamasını kapat (eğitim aşaması değil)
        output = evaluate(lineToTensor(input_line))  # Girdi satırını değerlendir

        # En yüksek N kategoriyi al
        topv, topi = output.topk(n_predictions, 1, True)  # En yüksek n_predictions değerini ve indekslerini al
        predictions = []  # Tahminleri saklamak için bir liste oluştur

        # Tahmin edilen kategorileri ve değerlerini yazdır
        for i in range(n_predictions):
            value = topv[0][i].item()  # Tahmin edilen değer
            category_index = topi[0][i].item()  # Tahmin edilen kategorinin indeksi
            print('(%.2f) %s' % (value, all_categories[category_index]))  # Değeri ve kategoriyi yazdır
            predictions.append([value, all_categories[category_index]])  # Tahminleri listeye ekle

# Örnek girdilerle tahminleri yap
predict('Dovesky')  # 'Dovesky' ismi için tahmin yap
predict('Jackson')  # 'Jackson' ismi için tahmin yap
predict('Satoshi')  # 'Satoshi' ismi için tahmin yap


The final versions of the scripts `in the Practical PyTorch
repo <https://github.com/spro/practical-pytorch/tree/master/char-rnn-classification>`__
split the above code into a few files:

-  ``data.py`` (loads files)
-  ``model.py`` (defines the RNN)
-  ``train.py`` (runs training)
-  ``predict.py`` (runs ``predict()`` with command line arguments)
-  ``server.py`` (serve prediction as a JSON API with bottle.py)

Run ``train.py`` to train and save the network.

Run ``predict.py`` with a name to view predictions:

::

    $ python predict.py Hazaki
    (-0.42) Japanese
    (-1.39) Polish
    (-3.51) Czech

Run ``server.py`` and visit http://localhost:5533/Yourname to get JSON
output of predictions.




Exercises
=========

-  Try with a different dataset of line -> category, for example:

   -  Any word -> language
   -  First name -> gender
   -  Character name -> writer
   -  Page title -> blog or subreddit

-  Get better results with a bigger and/or better shaped network

   -  Add more linear layers
   -  Try the ``nn.LSTM`` and ``nn.GRU`` layers
   -  Combine multiple of these RNNs as a higher level network


